In [1]:
from eval_metrics.glossary import *

import json
def load_jsonl(path):
    data=[]
    with open(path, 'r', encoding='utf-8') as reader:
        for line in reader:
            data.append(json.loads(line))
    return data 

ROOT_PATH = "/data/aofei"
dataset = "VQA_RAD"
lora_pred_path = "/data/aofei/hallucination/mitigation/VQA_RAD/llava_med/lora/epoch9/inference/pred.jsonl"
bv_pred_path = "/data/aofei/hallucination/mitigation/VQA_RAD/llava_med/moe_query/lora_0.15_bbox_0.1/epoch9/inference/pred.jsonl"

gt_path = f"{ROOT_PATH}/hallucination/{dataset}/data/test.json"

gt = json.load(open(gt_path, 'r'))
# candidate = json.load(open(args.candidate, 'r'))
lora_pred = load_jsonl(lora_pred_path)
bv_pred = load_jsonl(bv_pred_path)

gt_ids = [item['id'] for item in gt]
pred_ids = [item['question_id'] for item in lora_pred]
bv_pred_ids = [item['question_id'] for item in bv_pred]
num_gt_ids, num_pred_ids = len(gt_ids), len(pred_ids)
print(f'num_gt_ids: {num_gt_ids} || num_pred_ids: {num_pred_ids}')
# import pdb; pdb.set_trace()
assert gt_ids == pred_ids, "please make sure pred and gt are exactly matched"
assert gt_ids == bv_pred_ids, "please make sure pred and gt are exactly matched"

num_gt_ids: 451 || num_pred_ids: 451


In [2]:
# the original lora
num_close = 0
correct_ids, wrong_ids = [], []
for gt_item, pred_item in zip(gt, lora_pred):
    try:
        gt_results = gt_item['conversations']
    except:
        gt_results = gt_item['conversatons']
    gt_value = gt_results[1]['value'].lower()
    pred_value = pred_item['text'].lower()

    gt_value = normalize_word(gt_value)
    pred_value = normalize_word(pred_value)

    if gt_item['answer_type'] == 'CLOSED':
        num_close += 1
        _id = pred_item['question_id']
        if 'yes' in pred_value or 'no' in pred_value:
            if gt_value in pred_value:
                correct_ids.append(_id)
            else:
                wrong_ids.append(_id)
        # else:
        #     wrong_ids.append(_id)

In [3]:
len(wrong_ids), len(correct_ids)

(76, 176)

In [4]:
# our method, without MOE
num_close = 0
bv_correct_ids, bv_wrong_ids, bv_close_fail_ids = [], [], []
for gt_item, pred_item in zip(gt, bv_pred):
    try:
        gt_results = gt_item['conversations']
    except:
        gt_results = gt_item['conversatons']
    gt_value = gt_results[1]['value'].lower()
    pred_value = pred_item['text'].lower()

    gt_value = normalize_word(gt_value)
    pred_value = normalize_word(pred_value)

    if gt_item['answer_type'] == 'CLOSED':
        num_close += 1
        _id = pred_item['question_id']
        if 'yes' in pred_value or 'no' in pred_value:
            if gt_value in pred_value:
                bv_correct_ids.append(_id)
            else:
                bv_wrong_ids.append(_id)
        else:
            bv_close_fail_ids.append(_id)
        # else:
        #     nobbox_wrong_ids.append(_id)

In [5]:
len(bv_wrong_ids), len(bv_correct_ids), len(bv_close_fail_ids)

(77, 172, 5)

In [6]:
common_wrong_ids, bv_wrongs, lora_wrongs = [], [], []
for i in wrong_ids:
    if i in bv_wrong_ids:
        common_wrong_ids.append(i)
    else:
        lora_wrongs.append(i)
for i in bv_wrong_ids:
    if i in wrong_ids:
        continue
    else:
        bv_wrongs.append(i)

common_correct_ids, bv_corrects, lora_corrects = [], [], []
for i in correct_ids:
    if i in bv_correct_ids:
        common_correct_ids.append(i)
    else:
        lora_corrects.append(i)
for i in bv_correct_ids:
    if i in correct_ids:
        continue
    else:
        bv_corrects.append(i)
"correct:", len(lora_corrects), len(bv_corrects), "wrong:", len(lora_wrongs), len(bv_wrongs),

('correct:', 24, 20, 'wrong:', 21, 22)

In [7]:
gt[0]
gt_id_dict = dict()
for i in gt:
    gt_id_dict[i['id']] = i

In [9]:
# specific questions wrongly answered by our method
for i in bv_wrongs:
    print(gt_id_dict[i]["conversations"])

[{'from': 'human', 'value': 'Does this patient have evidence of pneumoperitoneum?'}, {'from': 'gpt', 'value': 'No'}]
[{'from': 'human', 'value': 'Is the cardiac silhouette less than half the diameter of the diaphragm?'}, {'from': 'gpt', 'value': 'Yes'}]
[{'from': 'human', 'value': 'Is the cardiac silhouette within normal size limits?'}, {'from': 'gpt', 'value': 'Yes'}]
[{'from': 'human', 'value': 'Does this look like a healthy liver?'}, {'from': 'gpt', 'value': 'Yes'}]
[{'from': 'human', 'value': 'Are lung markings present in the entire thoracic cavity?'}, {'from': 'gpt', 'value': 'Yes'}]
[{'from': 'human', 'value': 'Is the gallbladder present?'}, {'from': 'gpt', 'value': 'Yes'}]
[{'from': 'human', 'value': 'Are the borders of the heart clearly visualized?'}, {'from': 'gpt', 'value': 'yes'}]
[{'from': 'human', 'value': 'Can you see the esophagus in this image?'}, {'from': 'gpt', 'value': 'No'}]
[{'from': 'human', 'value': 'Is the mediastinum visualized?'}, {'from': 'gpt', 'value': 'No'

In [10]:
for i in lora_wrongs:
    print(gt_id_dict[i]["conversations"])

[{'from': 'human', 'value': 'Is there small bowel thickening present?'}, {'from': 'gpt', 'value': 'Yes'}]
[{'from': 'human', 'value': 'are there calcifications present on the abdominal aorta?'}, {'from': 'gpt', 'value': 'yes'}]
[{'from': 'human', 'value': 'Has the midline of the mediastinum shifted?'}, {'from': 'gpt', 'value': 'No'}]
[{'from': 'human', 'value': 'Is the liver normal?'}, {'from': 'gpt', 'value': 'Yes'}]
[{'from': 'human', 'value': 'Is this in a coronal plane?'}, {'from': 'gpt', 'value': 'no'}]
[{'from': 'human', 'value': 'Is there a pneumothorax present?'}, {'from': 'gpt', 'value': 'No'}]
[{'from': 'human', 'value': 'Is the heart wider than more than half the width of the thorax?'}, {'from': 'gpt', 'value': 'No'}]
[{'from': 'human', 'value': 'Is a pneumothorax present in the left apex?'}, {'from': 'gpt', 'value': 'No'}]
[{'from': 'human', 'value': 'Can fluids be highlighted with this modality?'}, {'from': 'gpt', 'value': 'yes'}]
[{'from': 'human', 'value': 'Can the later

In [12]:
bv_corrects
# gt_id_dict[11979]
for i in bv_corrects:
    print(gt_id_dict[i])

{'image': 'xmlab105/source.jpg', 'id': 11961, 'organ': 'Lung', 'answer_type': 'CLOSED', 'conversations': [{'from': 'human', 'value': '<image>\nDoes the picture contain heart?'}, {'from': 'gpt', 'value': 'No'}]}
{'image': 'xmlab299/source.jpg', 'id': 12273, 'organ': 'Lung', 'answer_type': 'CLOSED', 'conversations': [{'from': 'human', 'value': '<image>\nCan pneumothorax be observed on the lower left lung in this picture?'}, {'from': 'gpt', 'value': 'No'}]}
{'image': 'xmlab314/source.jpg', 'id': 12333, 'organ': 'Lung', 'answer_type': 'CLOSED', 'conversations': [{'from': 'human', 'value': '<image>\nCan pneumonia be observed on the right lung in this picture?'}, {'from': 'gpt', 'value': 'Yes'}]}
{'image': 'xmlab361/source.jpg', 'id': 12438, 'organ': 'Lung', 'answer_type': 'CLOSED', 'conversations': [{'from': 'human', 'value': '<image>\nCan nodule be observed on the lower right lung in this picture?'}, {'from': 'gpt', 'value': 'No'}]}
{'image': 'xmlab378/source.jpg', 'id': 12496, 'organ': 'L

In [11]:
wrong_after_tuning_ids = list(set(lora_corrects) and set(bv_wrongs))
print(len(wrong_after_tuning_ids), len(lora_corrects), len(bv_wrongs))

correct_after_tuning_ids = list(set(lora_wrongs) and set(bv_corrects))
len(correct_after_tuning_ids), len(lora_wrongs), len(bv_corrects)

22 24 22


(20, 21, 20)

In [12]:
# wrong_after_tuning_ids
for i in wrong_after_tuning_ids:
    print(gt_id_dict[i])

{'conversations': [{'from': 'human', 'value': 'Is the gallbladder present?'}, {'from': 'gpt', 'value': 'Yes'}], 'answer_type': 'CLOSED', 'image': 'synpic24220.jpg', 'id': 391, 'text': 'Is the gallbladder present?'}
{'conversations': [{'from': 'human', 'value': "Can the patient's encephalon be seen in this image?"}, {'from': 'gpt', 'value': 'No'}], 'answer_type': 'CLOSED', 'image': 'synpic24878.jpg', 'id': 1544, 'text': "Can the patient's encephalon be seen in this image?"}
{'conversations': [{'from': 'human', 'value': 'is there hylar lymphadenopathy?'}, {'from': 'gpt', 'value': 'yes'}], 'answer_type': 'CLOSED', 'image': 'synpic32962.jpg', 'id': 649, 'text': 'is there hylar lymphadenopathy?'}
{'conversations': [{'from': 'human', 'value': 'Is pneumo intestinalis present?'}, {'from': 'gpt', 'value': 'yes'}], 'answer_type': 'CLOSED', 'image': 'synpic23008.jpg', 'id': 1555, 'text': 'Is pneumo intestinalis present?'}
{'conversations': [{'from': 'human', 'value': 'Does this patient have a pne

In [13]:
# correct_after_tuning_ids
for i in correct_after_tuning_ids:
    print(gt_id_dict[i])

{'conversations': [{'from': 'human', 'value': 'Do you see hylar lymphadenopathy in this image?'}, {'from': 'gpt', 'value': 'yes'}], 'answer_type': 'CLOSED', 'image': 'synpic32962.jpg', 'id': 648, 'text': 'Do you see hylar lymphadenopathy in this image?'}
{'conversations': [{'from': 'human', 'value': 'Is there cardiomegaly?'}, {'from': 'gpt', 'value': 'yes'}], 'answer_type': 'CLOSED', 'image': 'synpic18651.jpg', 'id': 655, 'text': 'Is there cardiomegaly?'}
{'conversations': [{'from': 'human', 'value': 'Is there air trapped in the body?'}, {'from': 'gpt', 'value': 'No'}], 'answer_type': 'CLOSED', 'image': 'synpic34054.jpg', 'id': 1297, 'text': 'Is there air trapped in the body?'}
{'conversations': [{'from': 'human', 'value': 'Are the ventricles an abnormal size?'}, {'from': 'gpt', 'value': 'no'}], 'answer_type': 'CLOSED', 'image': 'synpic31928.jpg', 'id': 1945, 'text': 'Are the ventricles an abnormal size?'}
{'conversations': [{'from': 'human', 'value': 'Is the trachea deviated in this i

In [ ]:
'Do you see hylar lymphadenopathy in this image?'
'Is there cardiomegaly?'
'Is there air trapped in the body?'
'Are the ventricles an abnormal size?'
'Is the trachea deviated in this image?'
'Has the midline of the mediastinum shifted?'
'Is the liver normal?'
'Is there small bowel thickening present?'
'Is this in a coronal plane?'
'Is there hilar lymphadenopathy?'
'Is a pneumothorax present in the left apex?'
'Is there a pneumothorax present?'
'Are nodules present in both lungs?'
'Can fluids be highlighted with this modality?'
'are there calcifications present on the abdominal aorta?'
'Can the lateral ventricles be seen?'
'Are there clearly defined cardiac borders in this image?'
'Is the heart wider than more than half the width of the thorax?'
'Is this a T1 weighted MRI?'
'Can a mass in the pancreatic head cause a liver injury?'